In [ ]:
import pandas as pd

# مثال لو اسم الملف:
DATA_PATH = "/content/Arabic_Reports(3000).csv"   # نفس الفولدر

df = pd.read_csv(DATA_PATH)
print(df.shape)
df.head()



In [ ]:
# توحيد أسماء الأعمدة
df = df.rename(columns={
    "البلاغ": "text",
    "التصنيف": "label"
})

df = df.dropna(subset=["text", "label"]).copy()
df["text"] = df["text"].astype(str)
df["label"] = df["label"].astype(str).str.strip()

df.head()


In [ ]:
label_map = {
    "Profile Hacking Identity Theft": 0,
    "EWallet Related Fraud": 1,
    "Fraud CallVishing": 2
}

id2label = {v: k for k, v in label_map.items()}

df["label_id"] = df["label"].map(label_map)
df["label_id"].value_counts()


In [ ]:
!pip install transformers accelerate torch


In [ ]:
import sys
print("Python exe:", sys.executable)
print("Python version:", sys.version)


In [ ]:
import sys
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install -U transformers accelerate torch


In [ ]:
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer, AutoModelForCausalLM
print("OK ✅")


In [ ]:
import sys
!{sys.executable} -m pip uninstall -y torchvision


In [ ]:
import torch
print("torch:", torch.__version__)
print("cuda in torch:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())


In [ ]:
import sys

# IMPORTANT: reinstall both torch and torchvision from the SAME cu128 wheel index
!{sys.executable} -m pip install -U --force-reinstall --no-cache-dir \
  torch torchvision --index-url https://download.pytorch.org/whl/cu128


In [ ]:
import sys
!{sys.executable} -m pip uninstall -y torchaudio


In [ ]:
import sys
!{sys.executable} -m pip install -U --force-reinstall --no-cache-dir "pillow==11.0.0"


In [ ]:
import PIL
print("Pillow:", PIL.__version__)

import torch
import torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("cuda:", torch.cuda.is_available())


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.float16
)

model.eval()
print("Loaded:", model_name)


In [ ]:
# ============================
# Inference + Show + Save
# (Model already loaded)
# ============================

import torch
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from IPython.display import display

# ---------- 1) Prepare Data ----------
df = df.rename(columns={"البلاغ": "text", "التصنيف": "label"}).copy()
df = df.dropna(subset=["text", "label"]).copy()
df["text"] = df["text"].astype(str)
df["label"] = df["label"].astype(str).str.strip()

# ---------- 2) Labels ----------
LABELS = [
    "Profile Hacking Identity Theft",
    "EWallet Related Fraud",
    "Fraud CallVishing"
]
label_map = {lab: i for i, lab in enumerate(LABELS)}

# ---------- 3) Classification Function ----------
def classify(text, max_new_tokens=12):
    prompt = f"""
Task: Text Classification

Choose ONLY ONE label:
- {LABELS[0]}
- {LABELS[1]}
- {LABELS[2]}

Rules:
- Output ONLY the label text.
- No explanation.

Report:
{text}

Label:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = decoded.split("Label:")[-1].strip().splitlines()[0]

    for lab in LABELS:
        if lab.lower() in pred.lower():
            return lab
    return "UNKNOWN"

# ---------- 4) Run Inference ----------
df_run = df.copy()
df_run["pred_label"] = [classify(t) for t in df_run["text"]]
df_run["pred_id"]   = df_run["pred_label"].map(label_map)
df_run["true_id"]   = df_run["label"].map(label_map)

# ---------- 5) DISPLAY RESULTS (قبل الحفظ) ----------
print("📊 Sample of results (first 20 rows):")
display(df_run.head(20))

# ---------- 6) Evaluation ----------
eval_df = df_run.dropna(subset=["true_id","pred_id"]).copy()

acc = accuracy_score(eval_df["true_id"], eval_df["pred_id"])
mf1 = f1_score(eval_df["true_id"], eval_df["pred_id"], average="macro")

print(f"\nAccuracy: {acc:.4f}")
print(f"Macro F1:  {mf1:.4f}\n")

print(classification_report(
    eval_df["true_id"],
    eval_df["pred_id"],
    target_names=LABELS,
    digits=4
))

# ---------- 7) Save ----------
out_path = "qwen3_8b_inference_results.csv"
df_run.to_csv(out_path, index=False, encoding="utf-8-sig")
print("💾 Saved:", out_path)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# قراءة الملف
df_kw = pd.read_csv("Arabic_Reports(3000).csv")

# توحيد الأعمدة
df_kw = df_kw.rename(columns={"البلاغ": "text", "التصنيف": "label"})
df_kw = df_kw.dropna(subset=["text", "label"]).copy()

# نركّز فقط على الاحتيال المالي
ewallet_df = df_kw[df_kw["label"] == "EWallet Related Fraud"]

# TF-IDF لاستخراج الكلمات الأهم
vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=30,
    stop_words=None
)

X = vectorizer.fit_transform(ewallet_df["text"])
keywords = vectorizer.get_feature_names_out()

print("🔑 Financial Fraud Keywords:")
print(keywords)


In [ ]:
# ============================
# Inference + Eval + Show + Save
# (Model already loaded)
# Choose prompt_mode: "balanced" or "fewshot"
# ============================

import torch
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from IPython.display import display

# ---------- Settings ----------
prompt_mode = "fewshot"   # "balanced" or "fewshot"
N = 300                   # جرّبي 300 ثم خليها len(df)
max_new_tokens = 8        # label فقط

# ---------- 1) Prepare Data ----------
df = df.rename(columns={"البلاغ": "text", "التصنيف": "label"}).copy()
df = df.dropna(subset=["text", "label"]).copy()
df["text"] = df["text"].astype(str)
df["label"] = df["label"].astype(str).str.strip()

# ---------- 2) Labels ----------
LABELS = [
    "Profile Hacking Identity Theft",
    "EWallet Related Fraud",
    "Fraud CallVishing"
]
label_map = {lab: i for i, lab in enumerate(LABELS)}

# ---------- 3) Prompts ----------
BALANCED_PROMPT = f"""
Task: Classify the Arabic report into EXACTLY ONE label.

Label A) {LABELS[0]}
Use when the report is about account hacking/compromise or identity theft:
- hacked account, account takeover, password reset, impersonation
- social media account hijack, profile access, stolen credentials

Label B) {LABELS[1]}
Use when the report is about financial fraud or unauthorized money movement:
- money deducted/withdrawn/transfer from account (من حسابي / خصم / تحويل)
- balance affected, amount stolen (مبلغ/المبلغ/المال/أموالي)
- payment/transfer via app or e-wallet services, payment links
- sharing payment/account numbers (رقم) in the context of transactions

Label C) {LABELS[2]}
Use when the report is phone-call based scam (vishing):
- scam call, impersonation via phone, caller asks for OTP/personal data
- “اتصلوا علي / مكالمة / رقم” with social engineering by phone

Rules:
- Output ONLY the label text exactly as written.
- No explanation.

Report:
{{text}}

Label:
""".strip()

FEWSHOT_PROMPT = f"""
Task: Classify the Arabic report into EXACTLY ONE label from:
- {LABELS[0]}
- {LABELS[1]}
- {LABELS[2]}

Examples:

Report: تم اختراق حسابي في انستقرام وتغيير كلمة المرور ودخل شخص ينتحل شخصيتي.
Label: {LABELS[0]}

Report: تم خصم مبلغ من حسابي عبر تطبيق دفع وتحويل بدون إذني وتضرر رصيدي.
Label: {LABELS[1]}

Report: اتصل علي شخص يدّعي أنه من البنك وطلب رمز التحقق OTP وأصر أعطيه بياناتي.
Label: {LABELS[2]}

Now classify this report.

Rules:
- Output ONLY the label text exactly as written.
- No explanation.

Report:
{{text}}

Label:
""".strip()

def build_prompt(text: str) -> str:
    if prompt_mode == "balanced":
        return BALANCED_PROMPT.format(text=text)
    elif prompt_mode == "fewshot":
        return FEWSHOT_PROMPT.format(text=text)
    else:
        raise ValueError("prompt_mode must be 'balanced' or 'fewshot'")

# ---------- 4) Classifier ----------
def classify(text: str) -> str:
    prompt = build_prompt(text)

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = decoded.split("Label:")[-1].strip().splitlines()[0].strip()

    # enforce allowed labels
    for lab in LABELS:
        if lab.lower() in pred.lower():
            return lab
    return "UNKNOWN"

# ---------- 5) Run ----------
df_run = df.iloc[:N].copy()
df_run["pred_label"] = [classify(t) for t in df_run["text"]]
df_run["pred_id"] = df_run["pred_label"].map(label_map)
df_run["true_id"] = df_run["label"].map(label_map)

# Show distribution (THIS IS IMPORTANT)
print("True label distribution:")
print(df_run["label"].value_counts(), "\n")

print("Pred label distribution:")
print(df_run["pred_label"].value_counts(), "\n")

# ---------- 6) Display sample ----------
print("📊 Sample results:")
display(df_run[["text", "label", "pred_label"]].head(20))

# ---------- 7) Eval ----------
eval_df = df_run.dropna(subset=["true_id","pred_id"]).copy()

acc = accuracy_score(eval_df["true_id"], eval_df["pred_id"])
mf1 = f1_score(eval_df["true_id"], eval_df["pred_id"], average="macro")

print(f"\nPrompt mode: {prompt_mode}")
print(f"Accuracy: {acc:.4f}")
print(f"Macro F1:  {mf1:.4f}\n")

print(classification_report(
    eval_df["true_id"], eval_df["pred_id"],
    target_names=LABELS,
    digits=4
))

# ---------- 8) Save ----------
out_path = f"qwen3_8b_{prompt_mode}_results.csv"
df_run.to_csv(out_path, index=False, encoding="utf-8-sig")
print("💾 Saved:", out_path)


In [ ]:
import re, math, torch, pandas as pd
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# =====================
# LOAD DATA
# =====================
DATA_PATH = "Arabic_Reports(3000).csv"
TEXT_COL  = "البلاغ"
LABEL_COL = "التصنيف"

df = pd.read_csv(DATA_PATH).dropna(subset=[TEXT_COL, LABEL_COL]).copy()

CLASSES = sorted(df[LABEL_COL].unique().tolist())

# =====================
# Arabic normalization
# =====================
def norm_ar(t: str) -> str:
    t = str(t).lower()
    t = re.sub(r"[\u064B-\u065F\u0670]", "", t)
    t = re.sub(r"[إأآا]", "ا", t)
    t = t.replace("ى", "ي").replace("ة", "ه")
    t = t.replace("ؤ", "و").replace("ئ", "ي")
    t = re.sub(r"[^\w\s]", " ", t)
    return re.sub(r"\s+", " ", t).strip()

df["_norm"] = df[TEXT_COL].apply(norm_ar)

# =====================
# BUILD KB FROM DATA (TF-IDF)
# =====================
TOP_WORDS = 40
TOP_BIGRAMS = 40

class_terms = {}

for cls in CLASSES:
    texts = df[df[LABEL_COL] == cls]["_norm"].tolist()

    vec = TfidfVectorizer(
        ngram_range=(1,2),
        max_features=TOP_WORDS + TOP_BIGRAMS
    )
    X = vec.fit_transform(texts)
    terms = vec.get_feature_names_out()

    weights = {}
    for term in terms:
        base = 2.0 if " " in term else 1.0
        weights[term] = base

    class_terms[cls] = weights

# =====================
# BOOST SIGNALS (خفيفة ومتوازنة)
# =====================
BOOST = {
    "Fraud CallVishing": ["مكالمة","اتصل","otp","رمز","كود","تحقق"],
    "EWallet Related Fraud": ["خصم","تحويل","سحب","مبلغ","رصيد","محفظة","مدى","ابل باي","stc pay","رابط دفع"],
    "Profile Hacking Identity Theft": ["اختراق","تهكير","تغيير كلمة المرور","حسابي","انتحال"]
}

for cls, terms in BOOST.items():
    for t in terms:
        t = norm_ar(t)
        class_terms[cls][t] = class_terms[cls].get(t, 0) + 3.0

# =====================
# KB Scoring
# =====================
def score_kb(text):
    t = norm_ar(text)
    scores = {}
    for cls, terms in class_terms.items():
        s = 0.0
        for term, w in terms.items():
            if term in t:
                s += w
        scores[cls] = s

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[0][0], ranked[1][0], ranked[0][1] - ranked[1][1]

# =====================
# Qwen Tie-breaker
# =====================
LABELS = CLASSES
label_set = set(LABELS)

def qwen_choose(text, a, b):
    prompt = f"""
Choose ONE label only:
- {a}
- {b}

Report:
{text}

Label:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k:v.to(model.device) for k,v in inputs.items()}

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=6, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("Label:")[-1].strip()

    return a if a.lower() in pred.lower() else b

# =====================
# RUN HYBRID
# =====================
MARGIN_TH = 2.0
preds = []

for text in tqdm(df[TEXT_COL], total=len(df)):
    b1, b2, margin = score_kb(text)
    if margin < MARGIN_TH:
        preds.append(qwen_choose(text, b1, b2))
    else:
        preds.append(b1)

df["pred_label"] = preds

# =====================
# EVAL
# =====================
acc = accuracy_score(df[LABEL_COL], df["pred_label"])
mf1 = f1_score(df[LABEL_COL], df["pred_label"], average="macro")

print(f"Accuracy:  {acc:.4f}")
print(f"Macro F1:  {mf1:.4f}\n")
print(classification_report(df[LABEL_COL], df["pred_label"], digits=4))

# =====================
# SAVE
# =====================
df.to_csv("hybrid_kb_from_data.csv", index=False, encoding="utf-8-sig")
print("Saved: hybrid_kb_from_data.csv")


In [ ]:
import re, math, torch, pandas as pd
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# =====================
# LOAD DATA
# =====================
DATA_PATH = "Arabic_Reports(3000).csv"
TEXT_COL  = "البلاغ"
LABEL_COL = "التصنيف"

df = pd.read_csv(DATA_PATH).dropna(subset=[TEXT_COL, LABEL_COL]).copy()

CLASSES = sorted(df[LABEL_COL].unique().tolist())

# =====================
# Arabic normalization
# =====================
def norm_ar(t: str) -> str:
    t = str(t).lower()
    t = re.sub(r"[\u064B-\u065F\u0670]", "", t)
    t = re.sub(r"[إأآا]", "ا", t)
    t = t.replace("ى", "ي").replace("ة", "ه")
    t = t.replace("ؤ", "و").replace("ئ", "ي")
    t = re.sub(r"[^\w\s]", " ", t)
    return re.sub(r"\s+", " ", t).strip()

df["_norm"] = df[TEXT_COL].apply(norm_ar)

# =====================
# BUILD KB FROM DATA (TF-IDF)
# =====================
TOP_WORDS = 40
TOP_BIGRAMS = 40

class_terms = {}

for cls in CLASSES:
    texts = df[df[LABEL_COL] == cls]["_norm"].tolist()

    vec = TfidfVectorizer(
        ngram_range=(1,2),
        max_features=TOP_WORDS + TOP_BIGRAMS
    )
    X = vec.fit_transform(texts)
    terms = vec.get_feature_names_out()

    weights = {}
    for term in terms:
        base = 2.0 if " " in term else 1.0
        weights[term] = base

    class_terms[cls] = weights

# =====================
# BOOST SIGNALS (خفيفة ومتوازنة)
# =====================
BOOST = {
    "Fraud CallVishing": ["مكالمة","اتصل","otp","رمز","كود","تحقق"],
    "EWallet Related Fraud": ["خصم","تحويل","سحب","مبلغ","رصيد","محفظة","مدى","ابل باي","stc pay","رابط دفع"],
    "Profile Hacking Identity Theft": ["اختراق","تهكير","تغيير كلمة المرور","حسابي","انتحال"]
}

for cls, terms in BOOST.items():
    for t in terms:
        t = norm_ar(t)
        class_terms[cls][t] = class_terms[cls].get(t, 0) + 3.0

# =====================
# KB Scoring
# =====================
def score_kb(text):
    t = norm_ar(text)
    scores = {}
    for cls, terms in class_terms.items():
        s = 0.0
        for term, w in terms.items():
            if term in t:
                s += w
        scores[cls] = s

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[0][0], ranked[1][0], ranked[0][1] - ranked[1][1]

# =====================
# Qwen Tie-breaker
# =====================
LABELS = CLASSES
label_set = set(LABELS)

def qwen_choose(text, a, b):
    prompt = f"""
Choose ONE label only:
- {a}
- {b}

Report:
{text}

Label:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k:v.to(model.device) for k,v in inputs.items()}

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=6, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("Label:")[-1].strip()

    return a if a.lower() in pred.lower() else b

# =====================
# RUN HYBRID
# =====================
MARGIN_TH = 2.0
preds = []

for text in tqdm(df[TEXT_COL], total=len(df)):
    b1, b2, margin = score_kb(text)
    if margin < MARGIN_TH:
        preds.append(qwen_choose(text, b1, b2))
    else:
        preds.append(b1)

df["pred_label"] = preds

# =====================
# EVAL
# =====================
acc = accuracy_score(df[LABEL_COL], df["pred_label"])
mf1 = f1_score(df[LABEL_COL], df["pred_label"], average="macro")

print(f"Accuracy:  {acc:.4f}")
print(f"Macro F1:  {mf1:.4f}\n")
print(classification_report(df[LABEL_COL], df["pred_label"], digits=4))

# =====================
# SAVE
# =====================
df.to_csv("hybrid_kb_from_data.csv", index=False, encoding="utf-8-sig")
print("Saved: hybrid_kb_from_data.csv")


In [ ]:
import re, torch, pandas as pd
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# =====================
# LOAD DATA
# =====================
DATA_PATH = "Arabic_Reports(3000).csv"
TEXT_COL  = "البلاغ"
LABEL_COL = "التصنيف"

df = pd.read_csv(DATA_PATH).dropna(subset=[TEXT_COL, LABEL_COL]).copy()
CLASSES = sorted(df[LABEL_COL].unique().tolist())

# =====================
# Arabic normalization & cleaning
# =====================
def norm_ar(t: str) -> str:
    t = str(t).lower()
    t = re.sub(r"[\u064B-\u065F\u0670]", "", t)
    t = re.sub(r"[إأآا]", "ا", t)
    t = t.replace("ى", "ي").replace("ة", "ه")
    t = t.replace("ؤ", "و").replace("ئ", "ي")
    t = re.sub(r"[^\w\s]", " ", t)
    t = re.sub(r"_x000d_", " ", t)  # remove extra artifacts
    t = re.sub(r"\s+", " ", t)
    return t.strip()

df["_norm"] = df[TEXT_COL].apply(norm_ar)

# =====================
# BUILD KB FROM DATA (TF-IDF)
# =====================
TOP_WORDS = 40
TOP_BIGRAMS = 40

class_terms = {}
for cls in CLASSES:
    texts = df[df[LABEL_COL] == cls]["_norm"].tolist()
    vec = TfidfVectorizer(ngram_range=(1,2), max_features=TOP_WORDS + TOP_BIGRAMS)
    X = vec.fit_transform(texts)
    terms = vec.get_feature_names_out()

    weights = {}
    for term in terms:
        base = 2.0 if " " in term else 1.0
        weights[term] = base
    class_terms[cls] = weights

# =====================
# BOOST SIGNALS (خفيفة ومتوازنة)
# =====================
BOOST = {
    "Fraud CallVishing": ["مكالمة","اتصل","otp","رمز","كود","تحقق"],
    "EWallet Related Fraud": ["خصم","تحويل","سحب","مبلغ","رصيد","محفظة","مدى","ابل باي","stc pay","رابط دفع"],
    "Profile Hacking Identity Theft": ["اختراق","تهكير","تغيير كلمة المرور","حسابي","انتحال"]
}

BOOST_WEIGHT = 2.0  # تقليل الوزن ليكون متوازن

for cls, terms in BOOST.items():
    for t in terms:
        t = norm_ar(t)
        class_terms[cls][t] = class_terms[cls].get(t, 0) + BOOST_WEIGHT

# =====================
# DISPLAY TOP TERMS PER CLASS
# =====================
print("أهم الكلمات لكل فئة بعد Boost:")
for cls, terms in class_terms.items():
    sorted_terms = sorted(terms.items(), key=lambda x: x[1], reverse=True)[:15]
    print(f"\nأهم 15 كلمات لفئة: {cls}")
    for term, w in sorted_terms:
        print(f"{term}: {w}")

# =====================
# KB Scoring
# =====================
def score_kb(text):
    t = norm_ar(text)
    scores = {}
    for cls, terms in class_terms.items():
        s = sum(w for term, w in terms.items() if term in t)
        scores[cls] = s
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[0][0], ranked[1][0], ranked[0][1] - ranked[1][1]

# =====================
# Qwen Tie-breaker
# =====================
LABELS = CLASSES
label_set = set(LABELS)

def qwen_choose(text, a, b):
    prompt = f"""
Choose ONE label only:
- {a}
- {b}

Report:
{text}

Label:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k:v.to(model.device) for k,v in inputs.items()}

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=6, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("Label:")[-1].strip()
    return a if a.lower() in pred.lower() else b

# =====================
# RUN HYBRID
# =====================
MARGIN_TH = 2.0
preds = []

for text in tqdm(df[TEXT_COL], total=len(df)):
    b1, b2, margin = score_kb(text)
    if margin < MARGIN_TH:
        preds.append(qwen_choose(text, b1, b2))
    else:
        preds.append(b1)

df["pred_label"] = preds

# =====================
# EVAL
# =====================
acc = accuracy_score(df[LABEL_COL], df["pred_label"])
mf1 = f1_score(df[LABEL_COL], df["pred_label"], average="macro")

print(f"\nAccuracy:  {acc:.4f}")
print(f"Macro F1:  {mf1:.4f}\n")
print(classification_report(df[LABEL_COL], df["pred_label"], digits=4))

# =====================
# SAVE
# =====================
df.to_csv("hybrid_kb_improved_balanced.csv", index=False, encoding="utf-8-sig")
print("Saved: hybrid_kb_improved_balanced.csv")


In [ ]:
import re, torch, pandas as pd
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# =====================
# LOAD DATA
# =====================
DATA_PATH = "Arabic_Reports(3000).csv"
TEXT_COL  = "البلاغ"
LABEL_COL = "التصنيف"

df = pd.read_csv(DATA_PATH).dropna(subset=[TEXT_COL, LABEL_COL]).copy()
CLASSES = sorted(df[LABEL_COL].unique().tolist())

# =====================
# Arabic normalization
# =====================
def norm_ar(t: str) -> str:
    t = str(t).lower()
    t = re.sub(r"[\u064B-\u065F\u0670]", "", t)
    t = re.sub(r"[إأآا]", "ا", t)
    t = t.replace("ى", "ي").replace("ة", "ه")
    t = t.replace("ؤ", "و").replace("ئ", "ي")
    t = re.sub(r"[^\w\s]", " ", t)
    return re.sub(r"\s+", " ", t).strip()

df["_norm"] = df[TEXT_COL].apply(norm_ar)

# =====================
# BUILD KB FROM DATA (TF-IDF)
# =====================
TOP_WORDS = 100
TOP_BIGRAMS = 50

class_terms = {}

for cls in CLASSES:
    texts = df[df[LABEL_COL] == cls]["_norm"].tolist()
    vec = TfidfVectorizer(ngram_range=(1,3), max_features=TOP_WORDS + TOP_BIGRAMS)
    X = vec.fit_transform(texts)
    terms = vec.get_feature_names_out()

    weights = {}
    for term in terms:
        base = 2.0 if " " in term else 1.0
        weights[term] = base
    class_terms[cls] = weights

# =====================
# BOOST SIGNALS (خفيفة ومتوازنة)
# =====================
BOOST = {
    "Fraud CallVishing": ["مكالمة","اتصل","otp","رمز","كود","تحقق","whatsapp"],
    "EWallet Related Fraud": ["خصم","تحويل","سحب","مبلغ","رصيد","محفظه","مدي","ابل باي","stc pay","رابط دفع"],
    "Profile Hacking Identity Theft": ["اختراق","تهكير","تغيير كلمة المرور","حسابي","انتحال"]
}

# زيادة الوزن للفئة الضعيفة Fraud CallVishing
BOOST_WEIGHTS = {"Fraud CallVishing": 4.0, "EWallet Related Fraud": 3.0, "Profile Hacking Identity Theft": 3.0}

for cls, terms in BOOST.items():
    w = BOOST_WEIGHTS.get(cls, 3.0)
    for t in terms:
        t = norm_ar(t)
        class_terms[cls][t] = class_terms[cls].get(t, 0) + w

# =====================
# KB Scoring
# =====================
def score_kb(text):
    t = norm_ar(text)
    scores = {}
    for cls, terms in class_terms.items():
        s = 0.0
        for term, w in terms.items():
            if term in t:
                s += w
        scores[cls] = s
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[0][0], ranked[1][0], ranked[0][1] - ranked[1][1]

# =====================
# Qwen Tie-breaker
# =====================
LABELS = CLASSES
label_set = set(LABELS)

def qwen_choose(text, a, b):
    prompt = f"""
Choose ONE label only:
- {a}
- {b}

Report:
{text}

Label:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k:v.to(model.device) for k,v in inputs.items()}

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=6, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("Label:")[-1].strip()
    return a if a.lower() in pred.lower() else b

# =====================
# RUN HYBRID
# =====================
MARGIN_TH = 2.0
preds = []

for text in tqdm(df[TEXT_COL], total=len(df)):
    b1, b2, margin = score_kb(text)
    if margin < MARGIN_TH:
        preds.append(qwen_choose(text, b1, b2))
    else:
        preds.append(b1)

df["pred_label"] = preds

# =====================
# EVAL
# =====================
acc = accuracy_score(df[LABEL_COL], df["pred_label"])
mf1 = f1_score(df[LABEL_COL], df["pred_label"], average="macro")

print(f"Accuracy:  {acc:.4f}")
print(f"Macro F1:  {mf1:.4f}\n")
print(classification_report(df[LABEL_COL], df["pred_label"], digits=4))

# =====================
# SAVE
# =====================
df.to_csv("hybrid_kb_enhanced.csv", index=False, encoding="utf-8-sig")
print("Saved: hybrid_kb_enhanced.csv")


In [ ]:
# =====================
# عرض أهم الكلمات لكل فئة
# =====================
TOP_N = 15  # عدد الكلمات التي تريد عرضها لكل فئة

for cls, terms in class_terms.items():
    print(f"\nأهم {TOP_N} كلمات لفئة: {cls}")
    # ترتيب الكلمات حسب الوزن من الأعلى للأقل
    sorted_terms = sorted(terms.items(), key=lambda x: x[1], reverse=True)
    for term, weight in sorted_terms[:TOP_N]:
        print(f"{term}: {weight}")


In [ ]:
import re, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# =====================
# LOAD DATA
# =====================
DATA_PATH = "Arabic_Reports(3000).csv"
TEXT_COL  = "البلاغ"
LABEL_COL = "التصنيف"

df = pd.read_csv(DATA_PATH).dropna(subset=[TEXT_COL, LABEL_COL]).copy()
CLASSES = sorted(df[LABEL_COL].unique().tolist())

# =====================
# Arabic normalization
# =====================
def norm_ar(t: str) -> str:
    t = str(t).lower()
    t = re.sub(r"[\u064B-\u065F\u0670]", "", t)
    t = re.sub(r"[إأآا]", "ا", t)
    t = t.replace("ى", "ي").replace("ة", "ه")
    t = t.replace("ؤ", "و").replace("ئ", "ي")
    t = re.sub(r"[^\w\s]", " ", t)
    return re.sub(r"\s+", " ", t).strip()

df["_norm"] = df[TEXT_COL].apply(norm_ar)

# =====================
# Arabic stopwords & cleaning
# =====================
STOPWORDS = [
    "من", "على", "الى", "في", "عن", "ما", "لم", "لا", "الذي", "هذه", "ذلك",
    "ارجو", "منكم", "تم", "من", "خلال", "مع", "علي", "الى", "_x000d_"
]

def clean_terms(term_list):
    return [t for t in term_list if t not in STOPWORDS]

# =====================
# BUILD KB FROM DATA (TF-IDF)
# =====================
TOP_WORDS = 50  # الكلمات الأحادية
TOP_BIGRAMS = 50  # العبارات الثنائية

class_terms = {}

for cls in CLASSES:
    texts = df[df[LABEL_COL] == cls]["_norm"].tolist()

    vec = TfidfVectorizer(
        ngram_range=(1,2),
        max_features=TOP_WORDS + TOP_BIGRAMS
    )
    X = vec.fit_transform(texts)
    terms = clean_terms(vec.get_feature_names_out())

    # الوزن الأساسي: 2 للكلمات الثنائية، 1 للأحادية
    weights = {term: (2.0 if " " in term else 1.0) for term in terms}

    class_terms[cls] = weights

# =====================
# BOOST كلمات مهمة
# =====================
BOOST = {
    "Fraud CallVishing": ["مكالمة","اتصل","otp","رمز","كود","تحقق"],
    "EWallet Related Fraud": ["خصم","تحويل","سحب","مبلغ","رصيد","محفظة","مدى","ابل باي","stc pay","رابط دفع"],
    "Profile Hacking Identity Theft": ["اختراق","تهكير","تغيير كلمة المرور","حسابي","انتحال"]
}

for cls, terms in BOOST.items():
    for t in terms:
        t = norm_ar(t)
        class_terms[cls][t] = class_terms[cls].get(t, 0) + 3.0

# =====================
# عرض أهم الكلمات بعد التنظيف والـ Boost
# =====================
TOP_N = 15
for cls, terms in class_terms.items():
    print(f"\nأهم {TOP_N} كلمات لفئة: {cls}")
    sorted_terms = sorted(terms.items(), key=lambda x: x[1], reverse=True)
    for term, weight in sorted_terms[:TOP_N]:
        print(f"{term}: {weight}")
